In [1]:
%matplotlib inline
# Importing necessary packages:

import re
import os
from glob import glob

import math
import numpy as np
import datetime
import pandas as pd

import matplotlib.pyplot as plt

import scipy.integrate as integrate
import scipy.special as special

from sympy import elliptic_pi

import MDAnalysis as mda 
from MDAnalysis.coordinates.LAMMPS import DumpReader
from MDAnalysis import transformations as mdatransform
from MDAnalysis.analysis import diffusionmap, align, rms


from PipeLine import *

In [2]:
fnames = glob("../test/N1000*.bug.*")
fnames = PipeLine.file_reader(fnames)

Total number of files is  1
Path to the first pair of the sorted file:  ('../test/N1000epsilon5.0r8.0lz308.5sig1.0nc20824dt0.005bdump1000adump5000ens1.bug.lammpstrj', '../test/N1000epsilon5.0r8.0lz308.5sig1.0nc20824dt0.005bdump1000adump5000ens1.bug.data')


In [ ]:
%%time
geom = 'cylinder'
PipeLine.extract_trj_bug(fnames[0], geom)

In [3]:
def extract_all_trj(all_data, all_trj, geom, resid = 0, lastname='Crd'):
    """
    The function does all the analysis on the a whole trajectory (simulation),

    Parameters:
    fname (string): the name of trajectory or simulation.
    geom (string): the name of the geometry of the simulation box.

    Requirements:
    All the above-defined fucntions and classes, MDAnalysis.
    
    Caution:
    The histograms are for the cylindrical coordinate system in the geom=cylinder.
    For other goemteries you need to redfined them.
    """
    if geom == 'cylinder':
        print("Setting the name of analyze file...\n")
        today = datetime.date.today().strftime('%Y%m%d')
        cellAttrs = cellAttributes(all_trj,geometry=geom, splitter='lammpstrj')
        ensFname = cellAttrs.filename
        
        print("")
        print(ensFname+" is analyzing...")
        print("")
        
        TIME_UNIT = 1.0 # time unit = dmon * sqrt(mmon * epmon)
        LJ_NSTEP = cellAttrs.bdump # Sampling steps via dump command in Lammps
        LJ_DT = cellAttrs.dt # Timestep during the sampling process in Lammps
        SAMPLE_DT = LJ_NSTEP * LJ_DT * TIME_UNIT

        cell = mda.Universe(all_data, all_trj, format = 'AMIRLAMMPSDUMP', atom_style = "id resid type x y z", dt = SAMPLE_DT)
        crowds = cell.select_atoms('resid '+str(resid)) # resid 1 is the atoms creating the chain, resid 0 is crowders

        
        print ("Caution:The histograms are for the cylindrical coordinate system in the geom=cylinder. For other goemteries you need to redfined them.")
        # Monomer local (number) density: 
        # radial direction of the cylindrical coordinate system
        edge_fname = 'rEdges'+lastname
        #bin_size = cellAttrs.dcrowd / 2.
        bin_size = 0.1
        lmax = cellAttrs.dcyl / 2.
        redges, rhist_collectors = bin_ceate(ensFname, bin_size, 0.0, lmax, edge_fname)

        # z direction of the cylindrical coordinate system
        edge_fname = 'zEdges'+lastname
        #bin_size = cellAttrs.dcrowd / 2.
        bin_size = 0.2
        lmax = cellAttrs.lcyl / 2.
        zedges, zhist_collectors = bin_ceate(ensFname, bin_size, -1.0 * lmax, lmax, edge_fname)

        # theta of the cylindrical coordinate system
        edge_fname = 'thetaEdges'+lastname
        bin_size = np.degrees(np.pi/24) # in degrees
        thetaedges, thetahist_collectors = bin_ceate(ensFname, bin_size, -1*np.degrees(np.pi), np.degrees(np.pi)-bin_size, edge_fname)
        
        if rhist_collectors.any() != 0:
            raise Exception("The histogram collectors are not empty!")
        if zhist_collectors.any() != 0:
            raise Exception("The histogram collectors are not empty!")
        if thetahist_collectors.any() != 0:
            raise Exception("The histogram collectors are not empty!")
        
        for ts in cell.trajectory: # the length of the for loop is equal to number of snapshots (configurations or frames)
            #number density in the cell's frame of reference
            # histogram in r direction
            rcrowder = np.linalg.norm(crowds.positions[:,:2], axis = 1) # r component of position of each monomer
            dummy_hist, _ = np.histogram(rcrowder, redges)
            rhist_collectors = np.add(rhist_collectors, dummy_hist)

            # histogram in z direction
            zcrowder = crowds.positions[:,2] # z component of position of each monomer
            dummy_hist, _ = np.histogram(zcrowder, zedges)
            zhist_collectors = np.add(zhist_collectors, dummy_hist)

            # histogram in theta 
            theta = np.degrees(np.arctan2(crowds.positions[:,1], crowds.positions[:,0])) # in degrees
            dummy_hist, _ = np.histogram(theta, thetaedges)
            thetahist_collectors = np.add(thetahist_collectors, dummy_hist)

        np.savetxt(ensFname+'_rHists'+lastname+'.csv', rhist_collectors, delimiter = ',')
        np.savetxt(ensFname+'_zHists'+lastname+'.csv', zhist_collectors, delimiter = ',')
        np.savetxt(ensFname+'_thetaHists'+lastname+'.csv', thetahist_collectors, delimiter = ',')
        
        print('done.')
    
    else :
        raise ValueError("For other geomteries, the function has not written yet.")

In [6]:
geom = 'cylinder'
PipeLine.extract_all_trj(fnames[0][1],fnames[0][0], geom, resid = 1, lastname='Mon')

Setting the name of analyze file...

This is an ensemble-averaged file, so ensemble number: 0
This is NOT an ALL file, so segnum: 0

N1000epsilon5.0r8.0lz308.5sig1.0nc20824dt0.005bdump1000adump5000ens1.bug is analyzing...

Caution:The histograms are for the cylindrical coordinate system in the geom=cylinder. For other goemteries you need to redfined them.


KeyboardInterrupt: 

In [27]:
trj_files = glob("../test/N1000epsilon5.0r8.0lz308.5sig1.0nc20824dt0.005bdump1000adump5000ens1.bug.lammpstrj")
all_tuples = PipeLine.file_reader(trj_files,extensions=['.lammpstrj'])

data_file = glob("../test/N2000epsilon5.0r13.0lz431sig2.0nc27780dtbdump1000adump5000ens8.all.data")
all_data = PipeLine.file_reader(data_file,extensions=['all.data'])

Total number of files is  1
Path to the first pair of the sorted file:  ('../test/N1000epsilon5.0r8.0lz308.5sig1.0nc20824dt0.005bdump1000adump5000ens1.bug.lammpstrj',)
Total number of files is  1
Path to the first pair of the sorted file:  ('../test/N2000epsilon5.0r13.0lz431sig2.0nc27780dtbdump1000adump5000ens8.all.data',)


In [33]:
all_data[0][0]

'../test/N2000epsilon5.0r13.0lz431sig2.0nc27780dtbdump1000adump5000ens8.all.data'

In [40]:
cell = mda.Universe(all_data[0][0])#,format='LAMMPS')
crds = cell.select_atoms('resid 0')
crds.atoms

<AtomGroup with 27780 atoms>

In [10]:
geom = 'cylinder'
PipeLine.extract_all_trj(data_file[0],all_tuples[0][0], geom, resid = 1, lastname='Mon')

Setting the name of analyze file...

This is an ensemble-averaged file, so ensemble number: 0
This is NOT an ALL file, so segnum: 0

N1000epsilon5.0r8.0lz308.5sig1.0nc20824dt0.005bdump1000adump5000ens1.bug is analyzing...



ValueError: The topology and AMIRLAMMPSDUMP trajectory files don't have the same number of atoms!
Topology number of atoms 29780
Trajectory: ../test/N1000epsilon5.0r8.0lz308.5sig1.0nc20824dt0.005bdump1000adump5000ens1.bug.lammpstrj Number of atoms 1000